In [1]:
from selenium import webdriver
import time
from selenium.common.exceptions import NoSuchElementException

In [2]:
driver = webdriver.Chrome('/Users/mac/projects/ChromeWebDriver/chromedriver')

In [3]:
# driver.get('https://www.dabangapp.com/room/5e843a4d99a00c17e3bc85c4')

# 아파트
driver.get('https://www.dabangapp.com/room/5e8c1f6ef9ae1d50cbdf754b')

In [4]:
time.sleep(2)
post_type = driver.find_element_by_xpath('/html/body/div[1]/div/div[1]/div/ul/li[1]/p/span')
post_type = post_type.get_attribute('innerText')
post_type

'아파트'

In [5]:
# 상세 설명 보기
try:
    button = driver.find_element_by_xpath("/html/body/div[1]/div/div[4]/div/div/button")
    driver.execute_script("arguments[0].click();", button)
except NoSuchElementException:
    pass


In [6]:
unrefined_description = driver.find_elements_by_xpath("/html/body/div[1]/div/div[4]/div/div")
description = unrefined_description[0].get_attribute("innerText")
description.replace("\n", "")

'깨끗함 입주일 5월4일'

In [7]:
unrefined_salesform = driver.find_elements_by_xpath('/html/body/div[1]/div/div[1]/div/ul/li[1]/div')
salesForm = unrefined_salesform[0].get_attribute("innerText")
salesForm = salesForm.replace('/', ' ')
salesForm = salesForm.replace('\n', '')
salesForm = salesForm.split()
salesType = salesForm[0]  # sales type
print('salesType: 매물 종류는>>', salesType)
salesDepositChar = salesForm[1]
if salesDepositChar.find('원'):
    salesDepositChar = salesDepositChar.replace('원', '')
    print('salesDepositChar: ', salesDepositChar)
salesdepositInt = salesDepositChar.replace('억', '00000000')
if salesdepositInt.find('만'):
    salesdepositInt = salesdepositInt.replace('만', '')
salesdepositInt = int(salesdepositInt)
print('salesdepositInt: ', salesdepositInt)


try:
    salesmonthlyChar = salesForm[2]
    print('salesmonthlyChar: ', salesmonthlyChar)
    salesmonthlyInt = salesmonthlyChar.replace('만원', '0000')
    salesmonthlyInt = int(salesmonthlyInt)
    print('salesmonthlyInt: ', salesmonthlyInt)
    if salesType == '전세':
        # 전세는 금액이 억, 만원이 붙어 있는 경우가 있어서 이렇게 처리.
        salesdepositInt = salesdepositInt + salesmonthlyInt
        salesDepositChar = salesDepositChar + salesmonthlyChar
except IndexError:
    salesmonthlyInt = 0
    salesmonthlyChar = ''

salesType: 매물 종류는>> 전세
salesDepositChar:  6억
salesdepositInt:  600000000


In [8]:
if post_type =="아파트":
    if salesType == "매매":
        address = driver.find_elements_by_xpath('/html/body/div[1]/div/div[5]/div[3]/div/p')
    else:
        address = driver.find_elements_by_xpath('/html/body/div[1]/div/div[5]/div[5]/div/p')
else:
    address = driver.find_elements_by_xpath('/html/body/div[1]/div/div[5]/div[4]/div/p')
if not address:
    address = driver.find_elements_by_xpath('/html/body/div[1]/div/div[5]/div/div/p')
print(address)
address = address[0].get_attribute('innerText')
address

[<selenium.webdriver.remote.webelement.WebElement (session="73a9ff731af52ffad75b6d36b392e714", element="0.005160792155824545-4")>, <selenium.webdriver.remote.webelement.WebElement (session="73a9ff731af52ffad75b6d36b392e714", element="0.005160792155824545-5")>]


'※ 방 수와 욕실 수 정보는 평면도와 다를 수 있습니다.'

In [9]:
unrefined_floor = driver.find_elements_by_xpath('/html/body/div[1]/div/div[2]/div/ul/li[1]/div')
total_floor = unrefined_floor[0].get_attribute('innerText')
total_floor = total_floor.split('/')
floor = total_floor[0]
floor = floor
print('floor>>', floor)
totalFloor = total_floor[1]
totalFloor = totalFloor.replace(' ', '')
totalFloor = totalFloor
print('totalFloor >> ', totalFloor)

floor>> 11층 
totalFloor >>  13층


In [10]:
unrefined_area = driver.find_elements_by_xpath('/html/body/div[1]/div/div[2]/div/ul/li[2]/div/span')
area = unrefined_area[0].get_attribute('innerText')
print('area >>', area)
areaChar = area
print('areaChar >> ', areaChar)

area >> 84.95 / 113.5㎡
areaChar >>  84.95 / 113.5㎡


In [11]:
# 평수로 변환해주는 버튼 클릭
time.sleep(1)
driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div/ul/li[2]/div/button').click()


In [12]:
unrefined_area = driver.find_elements_by_xpath('/html/body/div[1]/div/div[2]/div/ul/li[2]/div/span')
supplyAreaChar = unrefined_area[0].get_attribute('innerText')
print('supplyAreaChar >> ', supplyAreaChar)
supplyAreaInt = supplyAreaChar.split('/')
supplyAreaInt = supplyAreaInt[1].replace('평', '')

supplyAreaInt = supplyAreaInt.strip()

supplyAreaInt = int(supplyAreaInt)
print('supplyAreaInt >> ', supplyAreaInt)

supplyAreaChar >>  26 / 34평
supplyAreaInt >>  34


### 매매에서는 shortRent가 None

In [13]:
try:
    if post_type=='아파트':
        unrefined_shortRent = driver.find_elements_by_xpath(
            '/html/body/div[1]/div/div[5]/div[3]/div/table/tbody/tr/td[5]/p')
    else:
        unrefined_shortRent = driver.find_elements_by_xpath('/html/body/div[1]/div/div[5]/div[2]/div/table/tbody/tr/td[5]/p')
        if not unrefined_shortRent:
            unrefined_shortRent = driver.find_elements_by_xpath('/html/body/div[1]/div/div[5]/div[3]/div/table/tbody/tr/td[4]/p')
    shortRent = unrefined_shortRent[0].get_attribute('innerText')
    shortRent = shortRent

    if shortRent == '불가능':
        shortRent = False
    else:
        shortRent = True
        
except IndexError:
    # 매매에서는 단기임대가 없음. 
    shortRent=None
print('shortRent >>', shortRent)

shortRent >> True


## management가 안보이면 None

In [14]:
try:
    if post_type=="아파트":
        management = driver.find_element_by_xpath('/html/body/div[1]/div/div[5]/div[3]/div/table/tbody/tr/td[3]')
    else:
        if salesType == "매매":
            management = driver.find_element_by_xpath('/html/body/div[1]/div/div[5]/div[2]/div/table/tbody/tr/td[2]')
        else:
            management = driver.find_element_by_xpath('/html/body/div[1]/div/div[5]/div[2]/div/table/tbody/tr/td[3]')
except NoSuchElementException:
    management = None
#     management = driver.find_element_by_xpath('/html/body/div[1]/div/div[5]/div[2]/div/table/tbody/tr/td[3]')
try:
    management = management.get_attribute('innerText')
    management = management.replace('\n', '')
    management = management.replace(' ', '')
    management = management.replace('(', ' ')
    management = management.replace(')', ' ')
    management = management.replace(',', ' ')
    management = management.strip(' ')
    management = management.split(' ')
except AttributeError:
    pass

In [15]:
try:
    managementPay = management.pop(0)
    if managementPay.find('만원'):
        managementPay = managementPay.replace('만원', ' ')
        if managementPay == '없음':
            managementPay = 0
        managementPay = int(managementPay)
    else:
        managementPay = 0
    totalFee = managementPay
    print(totalFee)
    print(management)
except IndexError:
    pass
except AttributeError:
    pass
except NameError:
    pass

0
[]


In [16]:
try:
    if post_type =="아파트":
        if salesType =="매매":
            unrefined_parking = None
        else:
            unrefined_parking = driver.find_elements_by_xpath('/html/body/div[1]/div/div[5]/div[3]/div/table/tbody/tr/td[4]/p')
    else:
        if salesType == "매매":
        # 일반 주택 매매
            unrefined_parking = driver.find_elements_by_xpath("/html/body/div[1]/div/div[5]/div[2]/div/table/tbody/tr/td[3]")
        else:    
            unrefined_parking = driver.find_elements_by_xpath("/html/body/div[1]/div/div[5]/div[2]/div/table/tbody/tr/td[4]/p")
    parkingDetail = unrefined_parking[0].get_attribute('innerText')
except IndexError:
    unrefined_parking = driver.find_elements_by_xpath(
        '/html/body/div[1]/div/div[5]/div[3]/div/table/tbody/tr/td[4]/p'
    )
    parkingDetail = unrefined_parking[0].get_attribute('innerText')
except TypeError:
    parkingDetail = True

In [17]:
parkingDetail

'문의'

In [18]:
if parkingDetail == '불가':
    parkingTF = False
else:
    parkingTF = True
print('parkingTF >>', parkingTF)

parkingTF >> True


In [19]:
try:
    if not salesType == "매매":
        if post_type =="아파트":
            unrefined_living_expenses = driver.find_elements_by_xpath('/html/body/div[1]/div/div[5]/div[3]/div/div/div/label')
            unrefined_living_expenses_detail = driver.find_elements_by_xpath('/html/body/div[1]/div/div[5]/div[3]/div/div/div/span')
        else:
            try:
                unrefined_living_expenses = driver.find_elements_by_xpath('/html/body/div[1]/div/div[5]/div[2]/div/div/div/label')
                unrefined_living_expenses_detail = driver.find_elements_by_xpath('/html/body/div[1]/div/div[5]/div[2]/div/div/div/span')
            except NoSuchElementException:
                unrefined_living_expenses = driver.find_elements_by_xpath('/html/body/div[1]/div/div[5]/div[3]/div/div/div/label')
                unrefined_living_expenses_detail = driver.find_elements_by_xpath('/html/body/div[1]/div/div[5]/div[3]/div/table/tbody/tr/td[3]/p[2]/span')                
    else:
        living_expenses = None
        living_expenses_detail = None
except NoSuchElementException:
    pass

In [20]:
print(unrefined_living_expenses)
print(unrefined_living_expenses_detail)

[<selenium.webdriver.remote.webelement.WebElement (session="73a9ff731af52ffad75b6d36b392e714", element="0.005160792155824545-12")>]
[<selenium.webdriver.remote.webelement.WebElement (session="73a9ff731af52ffad75b6d36b392e714", element="0.005160792155824545-13")>]


In [21]:
try:
    unrefined_living_expenses = unrefined_living_expenses[0].get_attribute('innerText')
    print(unrefined_living_expenses)
    living_expenses = unrefined_living_expenses.replace(' ', '')
    living_expenses_detail = unrefined_living_expenses_detail[0].get_attribute('innerText')
    print(living_expenses_detail)
except IndexError:
    unrefined_living_expenses = driver.find_elements_by_xpath(
        '/html/body/div[1]/div/div[5]/div[3]/div/div/div/label'
    )
    unrefined_living_expenses_detail = driver.find_elements_by_xpath(
        '/html/body/div[1]/div/div[5]/div[3]/div/div/div/span')
    unrefined_living_expenses = unrefined_living_expenses[0].get_attribute('innerText')
    living_expenses = unrefined_living_expenses.replace(' ', '')
    living_expenses_detail = unrefined_living_expenses_detail[0].get_attribute('innerText')
except TypeError:
    pass

0만 원 + α
(-)


In [22]:
print('living_expenses >>', living_expenses)
print('living_expenses_detail >> ', living_expenses_detail)

living_expenses >> 0만원+α
living_expenses_detail >>  (-)
